In [1]:
%matplotlib nbagg
%load_ext line_profiler

In [2]:
import time
import logging
from contextlib import contextmanager

import numpy as np
import matplotlib.pyplot as plt

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
from libertem.corrections.coordinates import flip_y, rotate_deg, identity
from libertem.masks import circular
from libertem.common.container import MaskContainer

In [5]:
from libertem_live.detectors.merlin import MerlinDataSocket, ReaderPool
from libertem_live.detectors.merlin.dataset import LiveDataSet
from libertem_live.api import LiveContext
from libertem.viz.bqp import BQLivePlot

INFO:empyre:Imported EMPyRe V-0.3.1 GIT-e85a58daa6bbd861c3aa1fe26e1d609f376f1adc


In [6]:
from ptychography40.reconstruction.ssb import SSB_UDF, generate_masks
from ptychography40.reconstruction.common import wavelength, get_shifted

## Camera setup

In [7]:
data_socket = MerlinDataSocket()
pool = ReaderPool(backend=data_socket, pool_size=2)

In [20]:
ds = LiveDataSet(
    scan_size=(128, 128),
    data_socket=data_socket,
    pool=pool,
    frames_per_partition=2*128,
)
ds = ds.initialize(None)  # TODO: integrate into LiveContext.load or how it will be called

In [9]:
def make_medipix_setup(data_socket, pool):
    @contextmanager
    def medipix_setup(dataset, udfs):
        print("priming camera for acquisition")
        # TODO: medipix control socket commands go here?
        
        # mer.set('numframes', 1024)
        # mer.set(...)
        
        with data_socket:
            yield
        print("camera teardown")
    return medipix_setup

In [10]:
livectx = LiveContext(make_medipix_setup(data_socket, pool))

## SSB setup

In [11]:
ds_shape_sig, ds_shape_nav = ds.shape.sig, ds.shape.nav

# Acceleration voltage in keV
U = 300
rec_params = {
    "dtype": np.float32,
    "lamb": wavelength(U),
    "dpix": 12.7e-12,
    "semiconv": 22.1346e-3,  # 2020-05-18
    "semiconv_pix": 31,  # 2020-05-18
    # applied right to left
    "transformation": rotate_deg(88) @ flip_y(),
    "cx": 123,
    "cy": 126,
    "cutoff": 16,  # number of pixels: trotters smaller than this will be removed
}
cutoff_freq = np.float32('inf')

mask_params = {
    # Shape of the reconstructed area
    'reconstruct_shape': tuple(ds.shape.nav),
    # Shape of a detector frame
    'mask_shape': tuple(ds.shape.sig),
    # Use the faster shifting method to generate trotters
    'method': 'shift',
}

In [12]:
%%time
trotters = generate_masks(**rec_params, **mask_params)

CPU times: user 8.68 s, sys: 64.5 ms, total: 8.75 s
Wall time: 8.75 s


In [13]:
trotters

Format,coo
Data Type,float32
Shape,"(8320, 256, 256)"
nnz,2592705
Density,0.004754992631765513
Read-only,True
Size,69.2M
Storage ratio,0.0


In [14]:
mask_container = MaskContainer(
    mask_factories=lambda: trotters, dtype=trotters.dtype, count=trotters.shape[0]
)

In [15]:
ssb_udf = SSB_UDF(**rec_params, mask_container=mask_container)

## SSB on live data

In [16]:
from libertem.viz.bqp import BQLivePlot
p0 = BQLivePlot(ds, ssb_udf, channel="phase")
p1 = BQLivePlot(ds, ssb_udf, channel="amplitude")

In [17]:
p1.display()

Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

In [18]:
p0.display()

Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

In [21]:
livectx.run_udf(dataset=ds, udf=[ssb_udf], plots=[p0])

priming camera for acquisition
camera teardown


({'fourier': <BufferWrapper kind=single dtype=complex64 extra_shape=(128, 128)>,
  'complex': <BufferWrapper kind=single dtype=complex64 extra_shape=(128, 128)>,
  'amplitude': <BufferWrapper kind=single dtype=float32 extra_shape=(128, 128)>,
  'phase': <BufferWrapper kind=single dtype=float32 extra_shape=(128, 128)>},)

## Offline Data

In [ ]:
from libertem.api import Context

In [ ]:
from libertem.executor.inline import InlineJobExecutor

In [ ]:
ctx = Context(executor=InlineJobExecutor())

In [ ]:
ds_offline = ctx.load("mib", path="/cachedata/reference/MIB/20200518 165148/default.hdr")

In [ ]:
from libertem.udf.sum import SumUDF

In [ ]:
%%time
ctx.run_udf(dataset=ds_offline, udf=SumUDF())

In [ ]:
%%time
yarrrrrrr = ctx.run_udf(dataset=ds_offline, udf=ssb_udf)